In [ ]:
%matplotlib inline

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Input
from tensorflow.keras.losses import MeanSquaredError, BinaryCrossentropy, MeanAbsoluteError
from tensorflow.keras.optimizers import SGD

#JIT for the data generation.
from numba import jit

from IPython import display

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Data Gen

In [ ]:
@jit
def Kernel(x, x0):
    sigma = 0.8
    protonFraction = 0.4
    norm = protonFraction/(np.sqrt(2.*np.pi)*sigma)
    return(norm*np.exp(-(x - x0)**2./(2.*sigma**2.)))

In [ ]:
@jit
def test_data_gen(fakeKernel=False, sigma=0.4):
    A = 197
    yBeam = 5.36
    slope = 0.5
    sigmaEtas = 0.2
    
    # generate input data
    nBaryons = np.random.randint(0, 2*A)
    randX = np.random.uniform(0, 1, size=nBaryons)
    etasBaryon = 1./slope*np.arcsinh((2.*randX - 1)*np.sinh(slope*yBeam))
    etasArr = np.linspace(-6.4, 6.4, 128)
    dNBdetas = np.zeros(len(etasArr))
    norm = 1./(np.sqrt(2.*np.pi)*sigmaEtas)
    for iB in etasBaryon:
        dNBdetas += norm*np.exp(-(etasArr - iB)**2./(2.*sigmaEtas**2.))
    
    # generate test data with convolution with a kernel
    dNpdy = np.zeros(len(etasArr))
    detas = etasArr[1] - etasArr[0]
    for i in range(len(etasArr)):
        dNpdy[i] = sum(Kernel(etasArr, etasArr[i])*dNBdetas)*detas

    return(etasArr, dNBdetas, dNpdy)

In [ ]:
def generate_data(size=128):
    baryon = []
    proton = []
    
    for iev in range( size ):
        x, y1, y2 = test_data_gen()
        
        baryon.append(y1)
        proton.append(y2)
                
    return(np.array(baryon, dtype=np.float32), np.array(proton))

# Generator and Discriminator Models

In [ ]:
#Global Constants
BATCH_SIZE = 256

#Encoder
ENCODER_KERNEL_SIZE = 3
ENCODER_RELU_ALPHA = 0.2

#Decoder
DECODER_KERNEL_SIZE = 3
DECODER_RELU_ALPHA = 0.2

#Optimizer Parameters
BETA_1 = 0.9
BETA_2 = 0.999
EPSILON = 1e-07
LEARNING_RATE = 1e-4

In [ ]:
def discriminator_model(dimShape=(128,1)):
    model = Sequential([
        layers.Input(shape=dimShape),
        
    ], name="Discriminator")

In [ ]:
def autoencoder_model(dimShape=(128,1)):
    model = Sequential([        
        #Encoder
        layers.Input(shape=dimShape),
        
        layers.Conv1D(8, 3, padding="same", dilation_rate=2, activation="relu"),
        layers.MaxPool1D(),
        
        layers.Conv1D(4, 3, padding="same", dilation_rate=2, activation="relu"),
        layers.MaxPool1D(),
        
        layers.AveragePooling1D(),
        
        layers.Flatten(),
        
        layers.Dense(4),
        
        layers.Dense(64),
        
        layers.Reshape((16,4)),
        
        layers.Conv1DTranspose(4, 3, strides=2, padding="same", activation="relu"),
        
        layers.Conv1DTranspose(8, 5, strides=2, padding="same", activation="relu"),
        layers.Conv1DTranspose(16, 7, strides=2, padding="same", activation="relu"),
        
        layers.Conv1D(1, 1, padding="same", activation="sigmoid")
    ], name="Auto_Encoder")

    return(model)

In [ ]:
A = autoencoder_model()

A.summary()

# Training Functions

In [ ]:
def graph_samples(epoch=None, loss=None):
    display.clear_output(wait=True)
    if epoch:
        print("Epoch: " + str(epoch))

    samples = 4
    scalar = 40
    baryon, proton = generate_data(samples)
    fake_proton = A.predict(baryon)
    
    #tf.print(fake_proton)
    
    fig = plt.figure(constrained_layout=True, figsize=(25,5))
    ax_array = fig.subplots(1, samples, squeeze=True)
    for i in range(samples):
        ax_array[i].plot(np.arange(0, len(baryon[i]), 1, int ), baryon[i])
        ax_array[i].plot(np.arange(0, len(fake_proton[i]), 1, int ), fake_proton[i]*scalar)
        ax_array[i].plot(np.arange(0, len(proton[i]), 1, int ), proton[i])
    plt.show()
    
    if loss:
        #slope over half epochs
        plt.plot(np.arange(0, len(loss), 1, int ), loss)
        fig = plt.Figure()
        plt.show()

In [ ]:
loss_func = BinaryCrossentropy()

optimizer = Adam(learning_rate = LEARNING_RATE, beta_1=BETA_1, beta_2=BETA_2, epsilon=EPSILON)

In [ ]:
@tf.function
def train_step(real_baryon, real_proton):
    with tf.GradientTape() as tape:
        fake_proton = A(real_baryon)
        loss = loss_func(real_proton, fake_proton)
    
    grad = tape.gradient(loss, A.trainable_variables)
    optimizer.apply_gradients(zip(grad, A.trainable_variables))
    
    return loss

In [ ]:
def train(epochs):
    total_loss = []
    average_loss = []
    
    for e in range(epochs):
        baryon, proton = generate_data(size=BATCH_SIZE)
        loss = train_step(baryon, proton)
        
        total_loss.append(loss)
        
        average_loss.append((1/(e+1))*np.sum(total_loss))
        
        if (e%20) == 0:
            graph_samples(e, loss=average_loss)
    return np.array(total_loss)

# Test and Train Zone

In [ ]:
total_loss = train(100000)